In [9]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from omegaconf import OmegaConf
from lightning_pose.utils.scripts import get_imgaug_transform, get_dataset, get_data_module, get_loss_factories
from lightning_pose.utils.io import return_absolute_data_paths
import hydra
import os
import pandas as pd
import seaborn as sns
import yaml

In [10]:
base_config_dir = "/home/mattw/Dropbox/research-code/pose-estimation/configs"
base_save_dir = Path("/teamspace/studios/this_studio/lightning-pose/outputs/betta_ood_preds/")
# save_dir = os.path.join(base_save_dir, dataset_name)

In [11]:
loss_type = "pca_singleview"
dataset_names = [ # model names
#         "mirror-mouse", 
#         "mirror-fish",
#         "crim13",
#         "fly", 
#         "ibl-pupil", 
#     "ibl-fingers", 
#         "ibl-paw-2view-1",
#         "facemap",
    #"betta",
    "model_sup_1",
    "model_sup_2",
    "model_unsup_1",
    "model_unsup_2",
    #"model_unsup_1_median",
    #"model_unsup_2_median",
    "model_unsup_1_mean",
    "model_unsup_2_mean",
]
loss_name = "Pose PCA"
xlabel = "Fraction of Principal Components Kept"
var_explained = {}
for dataset_name in dataset_names:

    cfg_file = base_save_dir / dataset_name / "config.yaml"
    cfg = OmegaConf.load(cfg_file)
    cfg.training.imgaug = 'default'
    cfg.training.train_prob = 1
    cfg.training.val_prob = 0
    cfg.training.train_frames = 1
#     cfg.data.csv_file = 'CollectedData_full.csv'

    model_cfg = cfg.copy()
    model_cfg.model.losses_to_use = [loss_type]
    data_dir, video_dir = return_absolute_data_paths(data_cfg=model_cfg.data)
    imgaug_transform = get_imgaug_transform(cfg=model_cfg)
    dataset = get_dataset(cfg=model_cfg, data_dir=data_dir, imgaug_transform=imgaug_transform)
    data_module = get_data_module(cfg=model_cfg, dataset=dataset, video_dir=video_dir)
    data_module.setup()
    # compute pca params
    loss_factories = get_loss_factories(cfg=model_cfg, data_module=data_module)
    pca_loss = loss_factories["unsupervised"].loss_instance_dict[loss_type]
    # compute cumulative var explained
    cum_sum_explained_variance = np.cumsum(pca_loss.pca.pca_object.explained_variance_ratio_)
    var_explained[dataset_name] = cum_sum_explained_variance
    # running into weird errors, maybe this helps
    del dataset
    del data_module
    del pca_loss

using default image augmentation pipeline (resizing only)
Number of labeled images in the full dataset (train+val+test): 1224
Dataset splits -- train: 1223, val: 1, test: 0


dlopen libcuda.so failed. Please install GPU driver.

RuntimeError: Assert on "device_id == CPU_ONLY_DEVICE_ID || cuInitChecked()" failed: You are trying to create a GPU DALI pipeline, while CUDA is not available. Please install CUDA or set `device_id = None` in Pipeline constructor. If running inside Docker container, you may need to use  `--gpus` option.

In [ ]:
sns.set_style("white")

plt.figure(figsize=(4.5, 4.5))

fraction_to_keep = 0.99

for dataset_name in dataset_names:
    num_total_comps = var_explained[dataset_name].shape[0]
    if loss_type == "pca_singleview":
        xrange = np.arange(1, num_total_comps + 1) / (num_total_comps + 1)
        label = "%s (%iD)" % (dataset_name, num_total_comps)
    else:
        xrange = np.arange(1, num_total_comps + 1)
        label = "%s (%i views)" % (dataset_name, num_total_comps // 2)
    plt.plot(
        xrange, var_explained[dataset_name],
        label=label,
        marker="o", linewidth=2, markersize=4
    )

if loss_type == "pca_singleview":
    xmin, xmax = 0, 1
    textx, texty = -0.04, fraction_to_keep-0.06
else:
    xmin, xmax = 0.5, 6
    textx, texty = 0.5, fraction_to_keep-0.04
    
plt.hlines(
    y=fraction_to_keep, 
    xmin=xmin, 
    xmax=xmax, 
    color = "black", linestyles="dashed", linewidth=2
)
plt.text(textx, texty, "0.99", fontsize=14)

plt.xlabel(xlabel, fontsize=16)
plt.ylabel("Cumulative Explained Variance", fontsize=16)
plt.title(f"{loss_name}", fontsize=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

# if loss_type == "Multiview PCA":
#     plt.xticks(np.arange(1, num_total_comps+1));

plt.legend(loc="lower right", fontsize=12)
plt.tight_layout()
# base_dir = '/home/mattw/Dropbox/research-text/papers/2023-litpose/figs-curr/fig2_litpose'
# plt.savefig(os.path.join(base_dir, '%s.pdf' % loss_type))
plt.show()